In [8]:
from Vector_Utils import *
import re

In [ ]:
prompt = "Global warming causes extinction"
document = Document("article.docx")
full_text = "\n".join([para.text for para in document.paragraphs])

In [10]:
prompt = llmd_prompt(prompt, full_text)

In [11]:
print("Prompt:", prompt)
emphasis_words = prompt.split()

Prompt: Global warming disrupts phytoplankton oxygen production, thaws permafrost releasing deadly diseases, triggers earthquakes, reduces cognitive ability, threatens food supply, and increases sea levels.


In [12]:
vectorizer = vectorizer()
prompt_vec = vectorizer.encode(prompt)

In [19]:
paragraphs = document.paragraphs
sentence_enders = re.compile(r'(?<=[.!?])\s+')

for paragraph in paragraphs:
    highest = 0.4
    best = ()

    sentences = re.split(sentence_enders, paragraph.text)

    if not vectorizer.screen_paragraph(sentences, prompt_vec, threshold=0.4):
        make_small(paragraph)
        continue

    if len(sentences) > 3:
        print("SPLIT")
        for i in range((len(sentences) // 3) + 1):
            k = i * 3
            if k + 3 <= len(sentences):
                sub_paragraph = " ".join(sentences[k:k+3])
            else:
                sub_paragraph = " ".join(sentences[k:])
            sub_highest = 0.4
            sub_best = ()
            clauses = clause_separator(sub_paragraph)
            for combination in ordered_combinations(clauses, round(len(clauses) * 0.8)):
                joined = " ".join(combination)
                vector = vectorizer.encode(joined)
                sim_score = vectorizer.similarity(vector, prompt_vec)
                if sim_score > sub_highest:
                    sub_highest = sim_score
                    sub_best = combination
            best += sub_best
    else:
        clauses = clause_separator(paragraph.text)
        for combination in ordered_combinations(clauses, round(len(clauses) * 0.8)):
            joined = " ".join(combination)
            vector = vectorizer.encode(joined)
            sim_score = vectorizer.similarity(vector, prompt_vec)
            if sim_score > highest:
                highest = sim_score
                best = combination

    print(highest, best)
    underline_best_match_in_paragraph(paragraph, best, emphasis_words)

document.save("article_annotated.docx")

0.1320258378982544
0.6043288707733154
0.6264033913612366 ('warming, by 2100 they might become hot enough to “stop oxygen production by phyto-plankton by disrupting the process of photosynthesis.” Given that two-thirds of the Earth’s oxygen comes from phytoplankton, that would “likely result in the mass mortality of animals and humans', '.”')
0.4421282112598419
SPLIT
0.4 ('A year later, above the Arctic Circle, in Siberia, a heat wave thawed a reindeer carcass that had been trapped', 'out, permafrost is', 'a “very good preserver of microbes and viruses', ', because it is', 'cold, there is no oxygen, and it is')
0.4406704902648926
SPLIT
0.4 ('this: as ice sheets melt', ', they take weight off land, and that can trigger', 'earthquakes — seismic activity is already increasing in Greenland and Alaska', '. Meanwhile, the added weight of the new seawater starts to bend the Earth’s crust. “That will give you a massive increase in volcanic activity.', 'It’ll activate faults to create', 'earthqu